### When using MPLs with multiple hidden layers, input data will be given to the neural network and then complex patterns within the input data are learnt through having multiple hidden layers. While this is good for learning complex patterns within the input layer, it is possible that we might miss the simple patterns within the input data that could have a huge impact in the accuracy of our prediction.

## Hence, what we can do is that we can use Functional API in order to generate a concatnate layer before the output layer that will receive the output of the last hidden layer and also the input layer and will fuse them together and gives it to the output layer.

#### We will use the california_housing dataset. I will run do the same preprocessing that I did in the regression with MLPs section ( just copy and pasting), and then will use Functional API to build the wide and deep model  

# Within this notebook, the following topics will be discussed:

    1- Different instances of wide and deep models 
    2- Functional APIs 
    3- Saving and loading models 
    4- Model checkpoint callback 
    5- EarlyStopping callback
    6- Creating a personal callback method 

In [57]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [58]:
california_housing= fetch_california_housing()

In [59]:
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [60]:
X_train0,X_test, y_train0, y_test= train_test_split(
    california_housing["data"],
    california_housing["target"])

In [61]:
X_train1, X_validation, y_train1, y_validation= train_test_split(X_train0,y_train0)

In [62]:
sc= StandardScaler()
X_train_s= sc.fit_transform(X_train1)
X_validation_s= sc.transform(X_validation)
X_test_s= sc.transform(X_test)

# Functional API

#### First we define the input layer of the model 

In [12]:
input_= keras.layers.Input(shape= X_train1.shape[1:]) # the input function has the shape attribute

##### The shape attribute the dimensionality (features) of each datapoint within the dataset 

In [13]:
X_train1.shape

(11610, 8)

11610 is the num of samples and 8 is the dimensionality 

In [14]:
X_train1.shape[1:]

(8,)

#### Defining the hidden layers 

In [15]:
hidden_layers1= keras.layers.Dense(50, activation= "relu")(input_)
hidden_layers2= keras.layers.Dense(10, activation= "relu")(hidden_layers1)

#### Defining the concatenate layer 

In [16]:
concatenate_layer= keras.layers.Concatenate()([input_, hidden_layers2])

#### Defining the output layer 

In [17]:
output= keras.layers.Dense(1)(concatenate_layer)

#### Defining the model by connecting the layers together 

In [18]:
model_f= keras.Model(inputs= [input_], outputs= [output])

## Compiling the model

In [19]:
model_f.compile(loss= "mean_squared_error", # the objective function that will be optimized by sgd
              optimizer= "sgd",
              metrics= ["mean_absolute_error"]) # metric that is used to represent the degree of accuracy of the model (MAE is used instead of MSE since the MSE is dollar powered by 2 !!)

## Fitting the model 

In [20]:
model_f.fit(X_train_s, y_train1, epochs= 30,
          validation_data= (X_validation_s,y_validation))

Epoch 1/30
204/363 [===============>..............] - ETA: 0s - loss: 0.8937 - mean_absolute_error: 0.6648 

2024-07-01 00:57:45.523621: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


363/363 [==============================] - 0s 441us/step - loss: 1.2549 - mean_absolute_error: 0.6555 - val_loss: 0.6047 - val_mean_absolute_error: 0.5560
Epoch 2/30
363/363 [==============================] - 0s 300us/step - loss: 1031.8739 - mean_absolute_error: 2.9139 - val_loss: 0.5689 - val_mean_absolute_error: 0.5590
Epoch 3/30
363/363 [==============================] - 0s 305us/step - loss: 0.5504 - mean_absolute_error: 0.5457 - val_loss: 0.5900 - val_mean_absolute_error: 0.5427
Epoch 4/30
363/363 [==============================] - 0s 315us/step - loss: 0.5359 - mean_absolute_error: 0.5345 - val_loss: 0.5289 - val_mean_absolute_error: 0.5378
Epoch 5/30
363/363 [==============================] - 0s 312us/step - loss: 0.6866 - mean_absolute_error: 0.5385 - val_loss: 0.5506 - val_mean_absolute_error: 0.5339
Epoch 6/30
363/363 [==============================] - 0s 299us/step - loss: 0.5355 - mean_absolute_error: 0.5338 - val_loss: 0.5491 - val_mean_absolute_error: 0.5317
Epoch 7/30
3

# Wide and deep models with two input sets 

Another posibility could be seperating our input into two sections, giving one section to the hidden layer 1 and the other section directly to the concatenate layer. 


Let's say our dataframe has 8 features. The way we seperate the input is that for example one section contains all samples but only has the values for the first 5 features and the other section has all samples but only has the values for the last three features. 


It is also important to note that it is possible to have sections of input that have overlapping features. For example, section one has the values for the feature 1 to feature 4 and section two is feature 2 to feature 8. 

### Preprocessing step 

In [21]:
california_housing= fetch_california_housing()

In [22]:
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [23]:
X_train0,X_test, y_train0, y_test= train_test_split(
    california_housing["data"],
    california_housing["target"])

In [24]:
X_train1, X_validation, y_train1, y_validation= train_test_split(X_train0,y_train0)

In [25]:
sc= StandardScaler()
X_train_s= sc.fit_transform(X_train1)
X_validation_s= sc.transform(X_validation)
X_test_s= sc.transform(X_test)

### Seperating the dataset into two different sections (cause we have two inputs) 

In [63]:
X_train_s_1, X_train_s_2= X_train_s[:, :6], X_train_s[:, -4:]
X_validation_s_1, X_validation_s_2= X_validation_s[:, :6], X_validation_s[:, -4:]
X_test_s_1, X_test_s_2= X_test_s[:, :6], X_test_s[:, -4:]

### Defining the model 

In [27]:
input_1= keras.layers.Input(shape= [6])
input_2= keras.layers.Input(shape= [4])
hidden_layers1= keras.layers.Dense(50, activation= "relu")(input_1)
hidden_layers2= keras.layers.Dense(10, activation= "relu")(hidden_layers1)
concatenate_layer= keras.layers.Concatenate()([input_2, hidden_layers2])
output= keras.layers.Dense(1)(concatenate_layer)
model_f2= keras.Model(inputs= [input_1,input_2], outputs= [output])


### Compiling the model 

In [28]:
model_f2.compile(loss= "mean_squared_error", # the objective function that will be optimized by sgd
              optimizer= "sgd",
              metrics= ["mean_absolute_error"]) # metric that is used to represent the degree of accuracy of the model (MAE is used instead of MSE since the MSE is dollar powered by 2 !!)

### Fitting the model

In [29]:
model_f2.fit((X_train_s_1, X_train_s_2), y_train1, epochs= 30,
          validation_data= ((X_validation_s_1, X_validation_s_2),
                            y_validation))

Epoch 1/30
363/363 [==============================] - 0s 433us/step - loss: 0.8644 - mean_absolute_error: 0.6380 - val_loss: 7.3953 - val_mean_absolute_error: 0.6102
Epoch 2/30
363/363 [==============================] - 0s 318us/step - loss: 0.8112 - mean_absolute_error: 0.5376 - val_loss: 0.4759 - val_mean_absolute_error: 0.5207
Epoch 3/30
363/363 [==============================] - 0s 316us/step - loss: 0.4739 - mean_absolute_error: 0.4985 - val_loss: 0.4506 - val_mean_absolute_error: 0.4918
Epoch 4/30
363/363 [==============================] - 0s 316us/step - loss: 0.4724 - mean_absolute_error: 0.4810 - val_loss: 0.5552 - val_mean_absolute_error: 0.4935
Epoch 5/30
363/363 [==============================] - 0s 314us/step - loss: 0.8345 - mean_absolute_error: 0.5182 - val_loss: 2.8009 - val_mean_absolute_error: 0.5168
Epoch 6/30
363/363 [==============================] - 0s 314us/step - loss: 2.4925 - mean_absolute_error: 0.4983 - val_loss: 94.9179 - val_mean_absolute_error: 0.8860
Epo

# Having deep and wide models with more than one output 

#### Useful for: 

        1- when having more than one output with different types ( one regression and one classification for example and the input is the same for example its just a picture 

        2- When having more than one output with the same type ( all is classification-based) 

        3- Helper out: having output for the hiiden layer(s) 

###  Lets say our model also has a Helper output as well as the main output 

In [66]:
input_1= keras.layers.Input(shape= [6])
input_2= keras.layers.Input(shape= [4])
hidden_layers1= keras.layers.Dense(50, activation= "relu")(input_1)
hidden_layers2= keras.layers.Dense(10, activation= "relu")(hidden_layers1)
concatenate_layer= keras.layers.Concatenate()([input_2, hidden_layers2])
output= keras.layers.Dense(1, name= "output")(concatenate_layer)
helper_output= keras.layers.Dense(1,name= "helper_output")(hidden_layers2)
model_sub= keras.Model(inputs= [input_1,input_2],
                   outputs= [output, helper_output])


In [67]:
model_sub.compile(loss= ["mean_squared_error", "mean_squared_error"],#first one for the main output, second one for the helper output 
              loss_weights= [0.8, 0.2],
              optimizer= "sgd",
              metrics= ["mean_absolute_error"]) 

1- Note that one loss is also defined for the helper output. the value of the helper output will be compared to the actual y for each data point to see how the model has done so far by the end of that hidden layer 

2- Note that weights are given to each loss. sum of these weights should be one. It is basically saying that the loss of the helper output has only 20 percent of interaction in updating the weights of the neural network , whereas the loss of the main output has 80 percent interaction 

In [68]:
model_sub.fit((X_train_s_1, X_train_s_2), (y_train1, y_train1), epochs=30,  # for the main output and the helper output we use y_train1 as the actual values
          validation_data=((X_validation_s_1, X_validation_s_2),
                           (y_validation, y_validation)))


Epoch 1/30
363/363 [==============================] - 0s 795us/step - loss: 0.8120 - output_loss: 0.6905 - helper_output_loss: 1.2979 - output_mean_absolute_error: 0.5928 - helper_output_mean_absolute_error: 0.7910 - val_loss: 0.5440 - val_output_loss: 0.5240 - val_helper_output_loss: 0.6240 - val_output_mean_absolute_error: 0.5252 - val_helper_output_mean_absolute_error: 0.5871
Epoch 2/30
363/363 [==============================] - 0s 397us/step - loss: 0.4890 - output_loss: 0.4698 - helper_output_loss: 0.5660 - output_mean_absolute_error: 0.4917 - helper_output_mean_absolute_error: 0.5543 - val_loss: 0.5275 - val_output_loss: 0.5168 - val_helper_output_loss: 0.5705 - val_output_mean_absolute_error: 0.5014 - val_helper_output_mean_absolute_error: 0.5561
Epoch 3/30
363/363 [==============================] - 0s 402us/step - loss: 1.1715 - output_loss: 1.3202 - helper_output_loss: 0.5765 - output_mean_absolute_error: 0.5424 - helper_output_mean_absolute_error: 0.5468 - val_loss: 1.1031 - 

# Model saving in Keras 

### Saving model_f 

In [33]:
model_f.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 50)           450         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 10)           510         ['dense[0][0]']                  
                                                                                                  
 concatenate (Concatenate)      (None, 18)           0           ['input_1[0][0]',                
                                                                  'dense_1[0][0]']            

In [34]:
model_f.save("housing_reg_model_f.h5")

### Loading model_f

In [35]:
model_f_reg = keras.models.load_model("housing_reg_model_f.h5")

In [36]:
model_f_reg.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 50)           450         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 10)           510         ['dense[0][0]']                  
                                                                                                  
 concatenate (Concatenate)      (None, 18)           0           ['input_1[0][0]',                
                                                                  'dense_1[0][0]']            

### Saving model_sub

In [37]:
model_sub.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 dense_6 (Dense)                (None, 50)           350         ['input_4[0][0]']                
                                                                                                  
 input_5 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense_7 (Dense)                (None, 10)           510         ['dense_6[0][0]']                
                                                                                            

In [38]:
model_sub.save("housing_reg_model_sub.h5")

In [39]:
model_sub_reg = keras.models.load_model("housing_reg_model_sub.h5")

In [40]:
model_sub.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 dense_6 (Dense)                (None, 50)           350         ['input_4[0][0]']                
                                                                                                  
 input_5 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense_7 (Dense)                (None, 10)           510         ['dense_6[0][0]']                
                                                                                            

# Callback 

### Model checkpoint callback: 

A model checkpoint callback is a mechanism in machine learning, specifically in neural network training, that helps to save the model at certain points during training. This is especially useful for preventing loss of progress in the event of interruptions and for maintaining the best version of the model. Here's a detailed explanation:

### What is a Model Checkpoint Callback?

A model checkpoint callback saves the model (or weights) to a file at certain intervals, such as at the end of an epoch or after a certain number of steps. The primary purposes of using a model checkpoint callback are:

1. **Preventing Loss of Progress**: If training is interrupted (e.g., due to hardware failure, power outage, or accidental termination), the model can be loaded from the last saved checkpoint and training can resume from that point.
2. **Saving the Best Model**: During training, the callback can monitor a specific metric (e.g., validation loss or accuracy) and save the model only when it performs better on that metric. This ensures that the best-performing model, according to the chosen metric, is saved.
3. **Periodic Saving**: Models can be saved periodically (e.g., every epoch) so that different stages of training can be analyzed or so that a recent state can be resumed in case of interruption.

The code you provided is an instance of the `ModelCheckpoint` callback in Keras, which is used to save the model during training. Here's a breakdown of each argument and its purpose:

1. **`filepath`**:
   - Specifies the path where the model file will be saved. The filename can contain placeholders such as `{epoch}` and `{val_loss:.2f}` to include the epoch number and validation loss in the filename.

2. **`monitor="val_loss"`**:
   - The metric to monitor for saving the model. In this case, it is set to monitor the validation loss. You can change this to other metrics like "val_accuracy" or "loss".

3. **`verbose=0`**:
   - Controls the verbosity of the output. If set to 0, no messages are printed. If set to 1, a message is printed each time the model is saved.

4. **`save_best_only=False`**:
   - If set to `True`, the model will only be saved when the monitored metric has improved. If `False`, the model is saved at the end of every epoch regardless of the metric's improvement.

5. **`save_weights_only=False`**:
   - If set to `True`, only the model's weights will be saved (using `model.save_weights(filepath)`). If `False`, the entire model is saved (using `model.save(filepath)`), which includes the architecture, optimizer, and state.

6. **`mode="auto"`**:
   - Specifies the mode for the monitoring metric. It can be "auto", "min", or "max". In "auto" mode, Keras will infer the mode from the name of the monitored metric (e.g., metrics ending in "acc" are inferred to be "max" mode, while metrics ending in "loss" are inferred to be "min" mode). "min" mode means the model will be saved when the monitored metric decreases, and "max" mode means the model will be saved when the monitored metric increases.

7. **`save_freq="epoch"`**:
   - Specifies when to save the model. The default value "epoch" means the model is saved at the end of every epoch. You can also set this to an integer value, which represents the number of samples between saves (e.g., `save_freq=1000` saves the model every 1000 samples).

8. **`initial_value_threshold=None`**:
   - Used to specify the initial value threshold for the monitored metric. If the monitored metric is above or below this threshold, the model will be saved. This is rarely used and is typically set to `None`.

### Using callback checkpoint for our model 

In [41]:
model_checkpoint_callback= keras.callbacks.ModelCheckpoint("model_cb_reg_housing.h5",
                                                          save_best_only= True)

**`"model_cb_reg_housing.h5"`**:
   - This is the `filepath` parameter. It specifies the path and filename where the model will be saved. In this case, the model will be saved as a file named `model_cb_reg_housing.h5`.


In [69]:
model_sub.fit((X_train_s_1, X_train_s_2), (y_train1, y_train1), epochs=30,
          validation_data=((X_validation_s_1, X_validation_s_2),
                           (y_validation, y_validation)),
             callbacks= [model_checkpoint_callback])

Epoch 1/30
363/363 [==============================] - 0s 480us/step - loss: 1.0713 - output_loss: 1.0095 - helper_output_loss: 1.3183 - output_mean_absolute_error: 0.7711 - helper_output_mean_absolute_error: 0.9051 - val_loss: 24.8786 - val_output_loss: 30.0011 - val_helper_output_loss: 4.3886 - val_output_mean_absolute_error: 0.8671 - val_helper_output_mean_absolute_error: 0.9418
Epoch 2/30
363/363 [==============================] - 0s 412us/step - loss: 1.0765 - output_loss: 1.0161 - helper_output_loss: 1.3183 - output_mean_absolute_error: 0.7676 - helper_output_mean_absolute_error: 0.9046 - val_loss: 24.4144 - val_output_loss: 29.4208 - val_helper_output_loss: 4.3885 - val_output_mean_absolute_error: 0.8954 - val_helper_output_mean_absolute_error: 0.9424
Epoch 3/30
363/363 [==============================] - 0s 402us/step - loss: 1.6211 - output_loss: 1.6969 - helper_output_loss: 1.3183 - output_mean_absolute_error: 0.7724 - helper_output_mean_absolute_error: 0.9050 - val_loss: 25.07

**`callbacks=[model_checkpoint_callback]`**:
   - This specifies the list of callback functions to be used during training. In this case, it includes `model_checkpoint_callback`, which is a `ModelCheckpoint` callback that saves the model during training under certain conditions.


Let's consider an example where you train a model to predict housing prices. The model takes two sets of features: one related to the location and another related to the property characteristics. During training, the `ModelCheckpoint` callback saves the model to `model_cb_reg_housing.h5` every time the validation loss decreases. After 30 epochs, you will have the best version of your model saved, which you can then use for making predictions on new data.

## EarlyStopping 

Early stopping is a powerful technique in Keras (and other machine learning frameworks) used to prevent overfitting and improve the generalization of your models. The EarlyStopping callback monitors a specific metric during training and stops the training process if the metric does not improve for a specified number of epochs, which is called the "patience" parameter. Here’s a detailed explanation:

### Key Concepts of EarlyStopping Callback

1. **Monitoring Metric**: 
   - The metric you want to monitor (e.g., validation loss, validation accuracy) is specified using the `monitor` parameter.
   - Commonly monitored metrics are `val_loss` (validation loss) and `val_accuracy` (validation accuracy).

2. **Patience**:
   - This parameter defines the number of epochs with no improvement after which training will be stopped.
   - For example, if `patience=5`, training will stop if there is no improvement in the monitored metric for 5 consecutive epochs.

3. **Mode**:
   - The `mode` parameter can be set to `'min'`, `'max'`, or `'auto'`.
   - `'min'` is used when the monitored metric should decrease (e.g., loss).
   - `'max'` is used when the monitored metric should increase (e.g., accuracy).
   - `'auto'` infers the mode from the name of the monitored metric.

4. **Restore Best Weights**:
   - When `restore_best_weights=True`, the model weights will be restored to the state of the best epoch before stopping the training. This is useful to ensure that the model has the best weights when training stops.

### Example Code

Here is a simple example of how to use the EarlyStopping callback in a Keras model:

```python
# EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',    # Metric to monitor
    patience=5,            # Number of epochs to wait before stopping
    mode='min',            # 'min' because we want to minimize validation loss
    restore_best_weights=True  # Restore best weights
)
```

### How It Works

- **Training Phase**: The model is trained on the training data, and after each epoch, the validation loss (or other monitored metric) is evaluated.
- **Monitoring Phase**: The EarlyStopping callback checks if the monitored metric has improved compared to the best value seen so far.
- **Patience Check**: If there is no improvement for a number of epochs equal to the `patience` parameter, training stops.
- **Best Weights Restoration**: If `restore_best_weights=True` is set, the model weights are reverted to those corresponding to the best value of the monitored metric.

### Using EarlyStopping and checkpoint callback as two callbacks for our Model

In [43]:
model_checkpoint_callback= keras.callbacks.ModelCheckpoint("model_cb_reg_housing.h5",
                                                          save_best_only= True)

earlystopping_callback= keras.callbacks.EarlyStopping(patience= 5,
                                                      restore_best_weights= True)

In [44]:
model_sub.fit((X_train_s_1, X_train_s_2), (y_train1, y_train1), epochs=200,
          validation_data=((X_validation_s_1, X_validation_s_2),
                           (y_validation, y_validation)),
             callbacks= [model_checkpoint_callback, earlystopping_callback])

Epoch 1/200
363/363 [==============================] - 0s 452us/step - loss: nan - output_loss: nan - helper_output_loss: nan - output_mean_absolute_error: nan - helper_output_mean_absolute_error: nan - val_loss: nan - val_output_loss: nan - val_helper_output_loss: nan - val_output_mean_absolute_error: nan - val_helper_output_mean_absolute_error: nan
Epoch 2/200
363/363 [==============================] - 0s 391us/step - loss: nan - output_loss: nan - helper_output_loss: nan - output_mean_absolute_error: nan - helper_output_mean_absolute_error: nan - val_loss: nan - val_output_loss: nan - val_helper_output_loss: nan - val_output_mean_absolute_error: nan - val_helper_output_mean_absolute_error: nan
Epoch 3/200
363/363 [==============================] - 0s 387us/step - loss: nan - output_loss: nan - helper_output_loss: nan - output_mean_absolute_error: nan - helper_output_mean_absolute_error: nan - val_loss: nan - val_output_loss: nan - val_helper_output_loss: nan - val_output_mean_absolu

## AS YOU CAN SEE, THE TRAINING HAS STOPPED AT EPOCH NUMBER 5 

1. **ModelCheckpoint**:
   - At the end of each epoch, the `ModelCheckpoint` callback checks if the monitored metric (by default, validation loss) has improved.
   - If `save_best_only=True` and the metric has improved, the model is saved to `"model_cb_reg_housing.h5"`.

2. **EarlyStopping**:
   - After each epoch, the `EarlyStopping` callback checks the monitored metric (by default, validation loss).
   - If the metric does not improve for 5 consecutive epochs (patience=5), training stops.
   - If `restore_best_weights=True`, the model weights are reverted to the best epoch where the monitored metric was the best.


Combining these two callbacks allows for efficient and effective model training:
- **ModelCheckpoint** ensures you save the best version of your model during training.
- **EarlyStopping** prevents unnecessary training epochs once the model stops improving, thus preventing overfitting and saving computational resources.


## Creating a personal callback method 

Engineers sometimes define their own callback methods in Keras or other deep learning frameworks for several reasons:

### 1. **Custom Requirements**:
   - **Specialized Monitoring**: Built-in callbacks may not support all the metrics or conditions that a specific project requires. Engineers can create custom callbacks to monitor and respond to custom metrics or specific intermediate results.
   - **Complex Conditions**: Projects might require more complex conditions for stopping training or saving models than what is provided by default. For instance, stopping training based on multiple conditions or a combination of metrics.

### 2. **Advanced Logging and Visualization**:
   - **Custom Logging**: Custom callbacks can be used to log additional information during training, such as specific layer outputs, gradients, or other intermediate values that are not tracked by default.
   - **Real-time Visualization**: Engineers might want to create custom visualizations, like plotting dynamic learning curves, visualizing activation maps, or other custom plots, during training.

### 3. **Automated Actions**:
   - **Adaptive Learning Rate**: While Keras provides callbacks for learning rate scheduling, custom callbacks can implement more sophisticated adaptive learning rate strategies tailored to the specific needs of the training process.
   - **Dynamic Architecture Adjustments**: In some cases, it may be beneficial to modify the model architecture dynamically during training (e.g., adding more neurons or layers based on performance metrics).

### 4. **Integration with External Tools**:
   - **Custom Notifications**: Engineers might need to integrate the training process with external systems, such as sending notifications or alerts (e.g., via email or messaging apps) when certain events occur during training.
   - **External Resource Management**: Custom callbacks can help manage external resources, like saving intermediate results to a database, interacting with cloud storage, or handling distributed training setups.

### 5. **Experimentation and Research**:
   - **Research and Experimentation**: Researchers often experiment with new training techniques, regularization methods, or optimization strategies that are not yet available as built-in callbacks. Custom callbacks allow for quick prototyping and experimentation.
   - **Hyperparameter Tuning**: Custom callbacks can be designed to modify hyperparameters on the fly based on intermediate training results, allowing for more dynamic and potentially more effective training processes.

In [45]:
class MyCallback(keras.callbacks.Callback):
    def on_train_end(self,logs= None):
        print("")
    def on_epoch_end(self, epoch, logs):
        print(logs["val_loss"])

In [46]:
mycb= MyCallback()

### Method Details

#### 1. `on_train_end(self, logs=None)`
```python
def on_train_end(self, logs=None):
    print("")
```
- **Purpose**: This method is called at the end of training.
- **Logs**: The `logs` parameter is a dictionary containing information about the training process, such as the final training and validation metrics.
- **Behavior**: In this implementation, it simply prints an empty string. This is a placeholder and can be modified to perform any action desired when the training ends.

#### 2. `on_epoch_end(self, epoch, logs)`
```python
def on_epoch_end(self, epoch, logs):
    print(logs["val_loss"])
```
- **Purpose**: This method is called at the end of each epoch.
- **Parameters**:
  - `epoch`: The index of the epoch that has just ended.
  - `logs`: A dictionary containing the logs for the epoch, including metrics like `val_loss`, `val_accuracy`, etc.
- **Behavior**: This implementation prints the validation loss (`val_loss`) at the end of each epoch. The `logs` dictionary contains the values of metrics computed during the epoch.

The `MyCallback` class is a simple example of creating a custom callback in Keras. It demonstrates how to extend the `keras.callbacks.Callback` class to monitor specific metrics and perform actions at specific points during the training process. This approach provides flexibility to meet unique requirements in model training and monitoring.

In [47]:
model_sub.fit((X_train_s_1, X_train_s_2), (y_train1, y_train1), epochs=200,
          validation_data=((X_validation_s_1, X_validation_s_2),
                           (y_validation, y_validation)),
             callbacks= [model_checkpoint_callback, earlystopping_callback,mycb])

Epoch 1/200
363/363 [==============================] - 0s 442us/step - loss: nan - output_loss: nan - helper_output_loss: nan - output_mean_absolute_error: nan - helper_output_mean_absolute_error: nan - val_loss: nan - val_output_loss: nan - val_helper_output_loss: nan - val_output_mean_absolute_error: nan - val_helper_output_mean_absolute_error: nan
Epoch 2/200
363/363 [==============================] - 0s 389us/step - loss: nan - output_loss: nan - helper_output_loss: nan - output_mean_absolute_error: nan - helper_output_mean_absolute_error: nan - val_loss: nan - val_output_loss: nan - val_helper_output_loss: nan - val_output_mean_absolute_error: nan - val_helper_output_mean_absolute_error: nan
Epoch 3/200
363/363 [==============================] - 0s 388us/step - loss: nan - output_loss: nan - helper_output_loss: nan - output_mean_absolute_error: nan - helper_output_mean_absolute_error: nan - val_loss: nan - val_output_loss: nan - val_helper_output_loss: nan - val_output_mean_absolu